In [1]:
!python /data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/tools/urdf_converter.py --xml_model_path 'legged_studio_a1_more_colli.urdf' --config_path 'a1'

I0203 17:52:02.068403 140608382158656 urdf_converter.py:58] Loading urdf model from legged_studio_a1_more_colli.urdf


In [104]:
# !python /data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/tools/mujoco_converter.py --xml_model_path 'a1_mjcf.txt' --config_path 'a1_mj'

In [2]:
from IPython.display import HTML
import jax
from jax import numpy as jp

import brax
from brax.v1.envs import env
from brax.v1.io import html
# from brax.envs import env
# from brax.io import html

In [10]:
with open('a1', 'r') as file:
    content = file.read()

In [11]:
class A1(env.Env):
    def __init__(self, system_config):
        super().__init__(config=system_config)

    def reset(self, rng: jp.ndarray) -> env.State:
        qp = self.sys.default_qp()
        pos = qp.pos
        info = self.sys.info(qp)
        obs = jp.zeros(1)
        reward, done, zero = jp.zeros(3)
        metrics = {
            'torsoIsUp': zero,
            'torsoHeight': zero
        }
        info = {'rng': rng}
        return env.State(qp, obs, reward, done, metrics, info)

    def step(self, state: env.State, action: jp.ndarray) -> env.State:
        qp, info = self.sys.step(state.qp, action)
        return state.replace(qp=qp)

In [12]:
a1 = A1(content)

KeyboardInterrupt: 

In [6]:
env_state = a1.reset(rng=jax.random.PRNGKey(0))
rollout = []

for i in range(10):
    print(i)
    rollout.append(env_state.qp)
    action = 10 * jp.sin(i / 100) * jp.ones(a1.action_size)
    env_state = a1.step(env_state, action)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


0


KeyboardInterrupt: 

In [143]:
# HTML(html.render(a1.sys, [a1.sys.default_qp()]))
HTML(html.render(a1.sys, rollout))